
<br>
RNN Practical — Intro to Recurrent Neural Networks<br>
Topics: Motivation, Basics, Architectures (One-to-Many, Many-to-One, etc.), Shared Parameters<br>
Instructions: Complete each task by filling in the "Your answer here" sections.<br>


In [14]:
import numpy as np

------------------------------<br>
Task 1: RNN Architectures <br>
------------------------------

In [15]:
def task1_architectures():
    """
    Identify the correct RNN architecture (One-to-One, One-to-Many, Many-to-One, Many-to-Many)
    for the following scenarios:
    a) Sentiment analysis of a sentence -> single label
    b) Music generation from a single start token -> output sequence
    c) Named entity recognition: tag each word in a sentence
    d) Machine translation: source sentence -> target sentence
    """
    # Your answer here:
    # a) Many-to-One
    # b) One-to-Many
    # c) Many-to-Many
    # d) Many-to-Many (encoder-decoder)

------------------------------<br>
Task 2: Shared Parameters <br>
------------------------------

In [16]:
def task2_shared_parameters():
    """
    Explain shared parameters in an RNN.
    Compute parameter counts for an example:
      input size d=4, hidden size h=3, sequence length T=10
    """
    # Your answer here:
    d = 4  # input size
    h = 3  # hidden size
    # Weights from input to hidden: d * h
    W_xh = d * h
    # Weights from hidden to hidden: h * h
    W_hh = h * h
    # Biases for hidden layer: h
    b_h = h
    total_params = W_xh + W_hh + b_h
    return total_params  # 12 + 9 + 3 = 24
print(task2_shared_parameters())  # Should print 24

24


------------------------------<br>
Task 3: Manual Forward Pass <br>
------------------------------

In [17]:
def task3_manual_forward_pass():
    """
    Compute hidden states manually for a small RNN using np.tanh.
    Input sequence length T=3, input size=3, hidden size=2
    """
    x_seq = [np.array([0.5, -1.0]), # this is seq of 3 inputs
             np.array([1.0, 0.0]),
             np.array([-0.5, 0.5])]
    h_prev = np.zeros(2) # initial hidden state with 0 and size 2
    W_xh = np.array([[0.6, -0.2], # weights from input to hidden
                     [0.1,  0.5]])
    W_hh = np.array([[0.3, 0.4], 
                     [-0.2, 0.2]])
    b_h = np.array([0.0,0.1])
    h_list = []

    # Your code here
    for x in x_seq:
        h_next = np.tanh(np.dot(W_xh, x) + np.dot(W_hh, h_prev) + b_h) # compute next hidden state
        h_list.append(h_next) # store it
        h_prev = h_next # update previous hidden state
    return h_list
print(task3_manual_forward_pass()) # Done By Renad , Samar and Dona not by AI 

[array([ 0.46211716, -0.33637554]), array([0.53994993, 0.04027965]), array([-0.21833125,  0.19743869])]


------------------------------<br>
Task 4: NumPy RNN Cell Implementation <br>
------------------------------

In [18]:
def task4_numpy_rnn_cell():
    """
    Implement a simple Many-to-One RNN in NumPy.
    Use rnn_forward to compute h_T, then compute a readout: y = W_hy h_T + b_y
    Predict class = argmax(y)
    """

    # Toy dataset
    toy_sequences = [
        [np.array([1.0,0.5]), np.array([0.2,0.1]), np.array([0.3,-0.1])],
        [np.array([-0.5,-0.4]), np.array([0.1,-0.2]), np.array([-0.3,-0.1])],
        [np.array([0.8,0.2]), np.array([0.5,0.4]), np.array([0.1,0.2])],
        [np.array([-0.6,-0.2]), np.array([-0.4,-0.3]), np.array([0.0,-0.1])]
    ]
    labels = np.array([1,0,1,0])

In [19]:
"""
Goal:
- Introduction to tensors in PyTorch
- Build a simple RNN-based classifier

Dataset:
- We will classify short sequences of numbers as "increasing" or "decreasing"
  Example:
    [1, 2, 3, 4] → Label: 1 (increasing)
    [5, 3, 1, 0] → Label: 0 (decreasing)
"""

import torch
import torch.nn as nn
import torch.optim as optim

# ====================================================
# STEP 1: Create a Tiny Synthetic Dataset
# ====================================================

def generate_data(num_samples=100, seq_len=4):
    X, y = [], []
    for _ in range(num_samples):
        if torch.rand(1).item() > 0.5:
            seq = torch.sort(torch.rand(seq_len))[0]   # Increasing
            label = 1
        else:
            seq = torch.sort(torch.rand(seq_len), descending=True)[0]  # Decreasing
            label = 0
        X.append(seq.unsqueeze(-1))  # (seq_len, input_size=1)
        y.append(label)
    return torch.stack(X), torch.tensor(y)

X, y = generate_data()
# X shape: (batch_size=100, seq_len=4, input_size=1)
# y shape: (batch_size=100)

# ====================================================
# STEP 2: Define a Simple RNN Classifier
# ====================================================

class RNNClassifier(nn.Module):
    def __init__(self, input_size=1, hidden_size=8, num_classes=2):
        super().__init__()
        self.rnn = nn.RNN(input_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        # x: (batch, seq_len, input_size)
        out, h = self.rnn(x)         # out: (batch, seq_len, hidden), h: (1, batch, hidden)
        last_hidden = h.squeeze(0)   # (batch, hidden_size)
        logits = self.fc(last_hidden) # (batch, num_classes)
        return logits

model = RNNClassifier()
print(model)

# ====================================================
# STEP 3: Train the Model
# ====================================================

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

num_epochs = 50
for epoch in range(num_epochs):
    logits = model(X)
    loss = criterion(logits, y)
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    if (epoch+1) % 10 == 0:
        pred = torch.argmax(logits, dim=1)
        acc = (pred == y).float().mean().item()
        print(f"Epoch [{epoch+1}/{num_epochs}] Loss: {loss.item():.4f} Acc: {acc:.2f}")

# ====================================================
# STEP 4: Test the Model on New Data
# ====================================================

test_X, test_y = generate_data(num_samples=10)
with torch.no_grad():
    test_logits = model(test_X)
    preds = torch.argmax(test_logits, dim=1)

print("\nPredictions vs Actual:")
for i in range(len(test_X)):
    print(f"Seq: {test_X[i].squeeze().tolist()} -> Pred: {preds[i].item()} | Actual: {test_y[i].item()}")


RNNClassifier(
  (rnn): RNN(1, 8, batch_first=True)
  (fc): Linear(in_features=8, out_features=2, bias=True)
)
Epoch [10/50] Loss: 0.6571 Acc: 0.54
Epoch [20/50] Loss: 0.5591 Acc: 0.97
Epoch [30/50] Loss: 0.3054 Acc: 1.00
Epoch [40/50] Loss: 0.1130 Acc: 0.96
Epoch [50/50] Loss: 0.0642 Acc: 0.99

Predictions vs Actual:
Seq: [0.8174512982368469, 0.49319136142730713, 0.4033474922180176, 0.24551081657409668] -> Pred: 0 | Actual: 0
Seq: [0.5612156391143799, 0.6108258962631226, 0.7026264071464539, 0.9758620262145996] -> Pred: 1 | Actual: 1
Seq: [0.9195457100868225, 0.7085305452346802, 0.6176550388336182, 0.32304489612579346] -> Pred: 0 | Actual: 0
Seq: [0.6818788647651672, 0.4330689311027527, 0.3951728343963623, 0.33381950855255127] -> Pred: 0 | Actual: 0
Seq: [0.2629486322402954, 0.35671699047088623, 0.506699800491333, 0.6426212191581726] -> Pred: 1 | Actual: 1
Seq: [0.36274904012680054, 0.4146166443824768, 0.958614706993103, 0.9708342552185059] -> Pred: 1 | Actual: 1
Seq: [0.96021085977554